# Introduction to Wireshark

#### 0. Pre-requisites
- Need native OS install for Wireless monitoring mode, virtual machines don't have access to hardware
- Need to check Network Interface Card chipset supports monitoring mode
- NOTE: Monitoring mode is otherwise known as *RFMON mode*
- If using virtual machines or an incompatible NIC you will need a USB WiFi card
- To test for chipset compatbility:
    - Windows: [hwinfo](https://www.hwinfo.com/) 
- Once you know the chipset, check compatibility here:
    - [Combatibility List](https://www.aircrack-ng.org/doku.php?id=compatibility_drivers&DokuWiki=01c89160d88dc29d91c1546165ca8089#compatibility)


#### 1. Topics/Sub-Topics

- Customizing Interface
- Network Interfaces
    - Promiscous Mode
        - What is promiscuous mode
    - Monitor Mode
        - What is monitor mode
            - In computer networking, promiscuous mode is a mode for a wired network interface controller (NIC) or wireless network interface controller (WNIC) that causes the controller to pass all traffic it receives to the central processing unit (CPU) rather than passing only the frames that the controller is specifically programmed to receive.
        - How to detect if your card supports monitor mode
- Profiles


- Filtering
    - Capture Filters
        - host 192.168.0.3
        - net 192.168.0.0/24 or src net 192.168.0.0/24
        - net 192.168.0.0 mask 255.255.255.0
        - dst net 192.168.0.0/24
        - src net 192.168
        - port 53
        - host www.example.com and not (port 80 or port 25)
        - host www.example.com and not port 80 and not port 25
        - port not 53 and not arp
        - not broadcast and not multicast
        - contains
    - Modifiers
        - ==
        - ++
        - ()
    - Common Protocols
        - http
        - udp
        - icmp
        - ftp
        - tcp
    - Advanced Filters
        - ip.addr
        - tcp.srcport
        - tcp.dstport
        - tcp.port
    - HTTP Packet Analysis
        - Frame
        - Ethernet II
            - MAC Address
                - Destination MAC (Router)
                - Source MAC (Machine)
        - Internet Protocol
            - IP Address
                - Destination IP (End Point)
                - Source IP (Machine)
        - Transmission Protocol
            - Source Port (Transmission Port)
            - Destination Port (End Point Connection)
        - HTTP
            - Packet Inspection
            - Cookies
            - HTTP Headers
    - Non-HTTPS traffic
        - Header Analysis
            - Can sniff HTTP packet and look at Authorization:Basic header to see username/password if being sent over unencrypted connection
    
    - Spot Suspicious Traffic On Your Network
        - If you have a port that is suspicious:
            - Ex Port 22: lsof -i :22
            ```
            COMMAND    PID   USER   FD   TYPE             DEVICE SIZE/OFF NODE NAME
Google    3666 jaywon  122u  IPv4 0x9d311f46dc5ea5c7      0t0  TCP 10.0.1.65:61415->151.101.0.175:https (ESTABLISHED)
            ```
    - GeoIP Database Import
    - Check for GeoIP support in Wireshark->About
    - Download free GeoLite version [here](http://geolite.maxmind.com/download/geoip/database/)
    - Go to Edit→Preferences→Name Resolution and select GeoIP database directories. Add the full path of each database directory.
    - Restart wireshark
    - Filter by Geo: `ip and not ip.geoip.country == "United States"`
    - Filter by Lat: `ip.geoip.lat > "66.5"`
    
    - Wireshark Command Line Interface
        - For use on headless servers
            - Can use CLI to capture traffic (.pcap files) and then load into GUI environment later (capture remote, analyze locally)
        - Basic commandline commands
            - `tshark -D` to list available interfaces
                - BEST PRACTICE: Use interface name and not number in list
            - `tshark -i [interfacename]` to start *analyzing* traffic (no save)
            - `tshark -i [interfacename] -i [interfacename]` to start sniffing traffic on multiple interfaces
            - `tshark -i [interfacename]`
        - Tshark File Output
            - Best Practice: Write files to `/tmp`, special permissions for anyone to write to
            - Capture stop conditions:
                - In `wireshark --help` allows you to control conditions for when captures should stop to not take up all space on disk
            - Usage with other commands:
                - Can do things like pipe capture to `mail` command, etc..
    - Network Card Modes
        - NOTE: ifconfig will show if a network card is in a given mode
        - NOTE: airmon-ng can show the chipset of the installed card as well as automatically enabling monitor mode. Will also show you any processes that may interfere with monitor mode. You may need to kill these processes. NetworkManager is a process that may cause problems since it auotmatically will try to connect to remembered hosts. Stop airmon-ng to turn off monitor mode.
        - Monitor Mode
            - IMPORTANT: Ensure your card supports Monitor Mode or you will need a USB Wireless card that does     
            - IMPORTANT: Make sure your wireless card is in monitor mode before trying to capture traffic
            - Monitor mode does not listen on an established connected but to general traffic on the network or available signals passing by
        - Example Setup:
            - `ifconfig 2wlps2s0 down` to bring down
            - Set wireless card to monitor mode `iwconfig 2lp2s0 mode monitor`
            - `ifconfig 2wlps2s0 up` to bring back up
            - `ifconfig` to ensure mode is set
        - Promiscuous Mode
            - Allow sniffing on local connected interface(s)
        - Master Mode
        - Ad-hoc Network Card Mode
            - Peer to peer
        - Mesh Network Mode
        - Repeater Mode
            - Set card to repeat all traffic, good for boosting weak wireless connection
    - Capturing Remote Traffic
        - Ensure with `airmon-ng check mon0` that no processes are interfering before you start
        - To scan available networks, run `airodump-ng mon0`
        - To start capturing a network from the output, run `airodump-ng mon0 --channel [CH] --bssid [BSSID] --write [nameoffileprefix]`
            - Output will have multiple formats (cap, csv, kismet.csv, kismet.netxml)
        - To read .cap files in wireshark GUI run `wireshark somefile.pcap`
    - Decryptying Captured Remote Traffic
        - Load up .cap file `wireshark somefile.pcap`
        - Edit->Preferences->Protocols->IEE 802.11
            - You must have the decryption key for the router whether it's SSL or username/pswd
            - Check Enable decryption
            - Click Encryption keys
                - In popup, select wpa-pwd or wpa-psk
                - Paste the key for psk (private key auth)
                - Enter username:password for pwd (password auth)
    - Wireshark and Nmap
        - Good practice in pentesting to test out nmap commands and see how much noise they generate as you determine what's the right mix for not setting off firewall alarms
        - Ways to cut down the number of requests:
            - Specify ports to scan rather than default 1000
                - target an individual ports will only send 2 packets on default scan
                    - `nmap 192.168.1.4 -p 80`
                - nmap -Pn scan to a single port will only send 1 request
                    - `nmap 192.168.1.4 -p 80 -Pn`
                - nmap -Pn scan with a few ports to reduce request number
                    - `nmap 192.168.1.4 -p 80,22 -Pn`
      
    - Alternatives
        - tcpdump
        - dumpcap
        - winpcap/libpcap
        - WinDump
        - Kismet
        - KisMac2
        - ....and more

#### 2. Pre-requisites
- Basic understanding of networks
- Basic understading of IP addresses and ports
- Basic understanding of packets
- If trying to snoop local virtual traffic ensure that Promiscuous Mode: Allow All is checked on bridge interface (VirtualBox)


#### 3. Desired Outcomes

#### 4. Resources
- http://chrissanders.org/captures/
- http://www.win10pcap.org/
- https://www.androidauthority.com/capture-data-open-wi-fi-726356/
- http://www.kalilinuxhack.com/2015/11/how-to-sniffing-network-using-kali-linux-2.0.html
- https://danielmiessler.com/study/tcpdump
- https://www.sans.org/reading-room/whitepapers/tools/extracting-files-network-packet-captures-36562


#### 5. Challenges/Exercises
    
    - Hands-on Exercise - Use Nmap and Wireshark to determine what is the volume of packets sent to a host to determine the stealthiest way to scan a network
    - Hands-on Exercise - Setup a telnet connection and sniff traffic to extract password
        - Filter on `telnet`
    - Hands-on Exercise - Setup an FTP server and connection and sniff traffic to extract password
        - Filter on `ftp`
    - Hands-on Exercise - Browse some HTTP/HTTPS sites and analyze traffic and extract files
        - Filter on `http`
        - Extract image by export raw data context menu on packet
    - Hands-on Exercise - Setup an HTTP server with Basic authentication and sniff traffic to show HTTP header information
    - Hands-on Exercise - Sniff traffic on SSH session and analyze the sequence of events.
        - Filter on `ssh`
        - Analyze sequences in handshake
    - Hands-on Exercise - Setup Monitor mode interface and inspect new 802.11 view
        - Filter by `wlan`
        - Filter by mac address with `wlan.addr == xx.xx.xx.xx.xx.xx`
        - https://wiki.wireshark.org/Wi-Fi
        - Analyze structure of a beacon frame?
        - Inspect channels, SSID, etc...
    - Hands-on Exercise - Setup SSH tunneling to Wireshark
    

#### 6. Self Study Objectives

#### 7. Estimated Time

#### 8. Applicable Standards


